In [ ]:
import time
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

sys.path.insert(0,"../")

from app.mlcode.predict_price_movements import CoinPricePredictor
from app.mlcode.determine_trading_state import DetermineTradingState
from app.mlcode.utils import read_in_data, read_in_yaml, running_on_aws, setup_logging, update_yaml_config

%pylab inline

import logging

from tqdm.notebook import tqdm
tqdm.pandas()

## Setup data
- All of these configs are stored in the notebooks folder

In [7]:
base_path = "configs/"
coin = "btc"
coin_to_predict = coin +"_"

constants = read_in_yaml( base_path + "constants.yml", running_on_aws())
    
trading_state_filename = base_path + coin_to_predict + "trading_state_config.yml"
trading_constants = read_in_yaml(trading_state_filename, running_on_aws())

won_lost_amount_filename = base_path + coin_to_predict + "won_and_lost_config.yml"
won_and_lost_amount_constants = read_in_yaml(won_lost_amount_filename, running_on_aws())
    

actions_to_take_filename =  base_path + coin_to_predict + "actions_to_take.yml"
actions_to_take_constants = read_in_yaml(actions_to_take_filename, running_on_aws())


ml_constants = read_in_yaml(base_path +constants["ml_config_filename"].split("/")[1], running_on_aws())

2022-04-01 09:12:53,478: : read_in_yaml  Reading in configs/constants.yml
2022-04-01 09:12:53,625: : read_in_yaml  Key = btc_product_code Value = BTC/USD
2022-04-01 09:12:53,627: : read_in_yaml  Key = tcn_modelname_btc Value = tcn_btc
2022-04-01 09:12:53,628: : read_in_yaml  Key = nbeats_modelname_btc Value = nbeats_btc
2022-04-01 09:12:53,629: : read_in_yaml  Key = bitcoin_csv_filename Value = tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv
2022-04-01 09:12:53,632: : read_in_yaml  Key = tcn_filename_btc Value = checkpoint_4499.pth.tar
2022-04-01 09:12:53,634: : read_in_yaml  Key = nbeats_filename_btc Value = checkpoint_257.pth.tar
2022-04-01 09:12:53,640: : read_in_yaml  Key = eth_product_code Value = ETH/USD
2022-04-01 09:12:53,643: : read_in_yaml  Key = tcn_modelname_eth Value = tcn_eth
2022-04-01 09:12:53,646: : read_in_yaml  Key = nbeats_modelname_eth Value = nbeats_eth
2022-04-01 09:12:53,648: : read_in_yaml  Key = etherum_csv_filename Value = tmp/historic_crypt

In [3]:
bitcoin_df = read_in_data("../tmp/" + constants["bitcoin_csv_filename"].split("/")[1], running_on_aws())
etherum_df = read_in_data("../tmp/" +constants["etherum_csv_filename"].split("/")[1], running_on_aws())
sol_df = read_in_data("../tmp/" +constants["sol_csv_filename"].split("/")[1], running_on_aws())
matic_df = read_in_data("../tmp/" +constants["matic_csv_filename"].split("/")[1], running_on_aws())
link_df = read_in_data("../tmp/" +constants["link_csv_filename"].split("/")[1], running_on_aws())

2022-03-30 21:55:26,970: : read_in_data  Input file ../tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv
2022-03-30 21:55:26,989: : read_in_data                 open     high      low    close       volume
date                                                       
2017-01-01   963.66  1003.08   958.70   998.33  147775008.0
2017-01-02   998.62  1031.39   996.70  1021.75  222184992.0
2017-01-03  1021.60  1044.08  1021.60  1043.84  185168000.0
2017-01-04  1044.40  1159.42  1044.40  1154.73  344945984.0
2017-01-05  1156.73  1191.10   910.42  1013.38  510199008.0
2022-03-30 21:55:27,002: : read_in_data                 open     high      low    close        volume
date                                                        
2022-03-26  44331.0  44792.0  44101.0  44531.0  2.278341e+07
2022-03-27  44531.0  46962.0  44448.0  46842.0  5.399869e+07
2022-03-28  46842.0  48220.0  46661.0  47160.0  8.354505e+07
2022-03-29  47160.0  48185.0  47001.0  47456.0  8.208843e+07
2022-03-30 

In [9]:
ml_constants

{'prediction_params': {'bollinger_window': 5,
  'no_of_std': 1.25,
  'lookback_window': [15, 30, 45],
  'prediction_n_days': 7,
  'model_name': ['TCN', 'NBEATS'],
  'work_dir': '/mnt/shared-storage'},
 'hyperparameters_tcn': {'dropout': 0.1,
  'random_state': 0,
  'dilation_base': 2,
  'kernel_size': 14,
  'num_filters': 7,
  'num_layers': 7,
  'weight_norm': True,
  'epochs': 55},
 'hyperparameters_nbeats': {'random_state': 0,
  'num_blocks': 5,
  'layer_widths': 256,
  'epochs': 10}}

 - BTC

In [5]:
predictor = CoinPricePredictor(
    coin, constants, ml_constants, bitcoin_df, additional_dfs=[etherum_df]  # spy_df
)

In [10]:
price_prediction = predictor.predict()

2022-04-01 09:31:20,600: : predict  Slicing dataframes
2022-04-01 09:31:20,697: : _slice_df  Slice date , earliest day of data for main df, = 2020-01-01 00:00:00
2022-04-01 09:31:20,877: : predict  Building Bollinger Bands
2022-04-01 09:31:21,005: : _build_technical_indicators  ---- Adding Bollinger Bands ----
2022-04-01 09:31:21,009: : _build_technical_indicators                 open     high      low    close        volume  Rolling Mean  \
date                                                                         
2022-03-26  44331.0  44792.0  44101.0  44531.0  2.278341e+07       43628.8   
2022-03-27  44531.0  46962.0  44448.0  46842.0  5.399869e+07       44521.8   
2022-03-28  46842.0  48220.0  46661.0  47160.0  8.354505e+07       45375.2   
2022-03-29  47160.0  48185.0  47001.0  47456.0  8.208843e+07       46064.0   
2022-03-30  47455.0  47716.0  46329.0  47085.0  5.690887e+07       46614.8   

            Bollinger High  Bollinger Low    stc      stoch        rsi  \
date       

  0%|          | 0/11 [00:00<?, ?it/s]

2022-04-01 09:31:22,149: : _train_models  Training nbeats


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

[2022-04-01 09:31:33,130] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-01 09:31:33,130] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-04-01 09:31:33,130: : _init_model  Time series values are 64-bits; casting model to float64.
[2022-04-01 09:31:33,635] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-01 09:31:33,635] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-04-01 09:31:33,635: : _init_model  Time series values are 64-bits; casting model to float64.
[2022-04-01 09:31:33,966] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-01 09:31:33,966] INFO | darts.models.forecasting.torch_forecasting_model | Time serie

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

KeyboardInterrupt: 